In [1]:
import logging
import os
import re
import shutil
import subprocess
import textwrap
import toml

from multiprocessing.pool import ThreadPool

logging.basicConfig(level=logging.INFO, format='%(asctime)s: %(threadName)s: %(message)s')

In [2]:
# Path to store package registry and the Git executable
packages_path = 'K:\\julia_packages'
git = 'K:\\Git\\bin\\git.exe'

In [3]:
# Set the URL to the Julia default package registry and clone/fetch the latest registry
reg_url = 'https://github.com/JuliaRegistries/General.git'
reg_path = os.path.join(packages_path, 'registries')
ret = None
if os.path.exists(reg_path):
    pwd = os.getcwd()
    os.chdir(reg_path)
    ret = os.system(f"{git} pull")
    os.chdir(pwd)
else:
    ret = os.system(f"{git} clone {reg_url} {reg_path}")
if ret != 0:
    raise RuntimeError("Clone of registries repository failed!")

In [4]:
# Parse the registry TOML file into a really big dictionary
with open(os.path.join(reg_path, 'Registry.toml')) as infile:
    registry_data = toml.load(infile)

In [5]:
# Walk through the TOML dictionary, grabbing the package names and their Git repo URLs
pkg_download = {}
for pkg in sorted(registry_data['packages'].values(), key=lambda x: x['name']):
    with open(os.path.join(reg_path, pkg['path'], 'Package.toml')) as infile:
        pkg_toml = toml.load(infile)
    pkg_repo = pkg_toml['repo'].replace('git://', 'https://')
    pkg_download[pkg_toml['name']] = pkg_repo

In [6]:
def clone_bare(pkg_repo, packages_path=packages_path, redo=False):

    """Performs a bare Git clone of a repository.
    
    Parameters
    ----------
    pkg_repo : str
      The URL of the Git repository to clone.
    packages_path : str
      The path to the local directory to clone into.
    redo : bool
      If a clone exists, delete it and re-clone, otherwise do nothing.
      
    Returns
    -------
    pkg_repo : str
      The input `pkg_repo` parameter.
    ret : int
      The return code from Git, or ``None`` if `redo` was ``False`` and the
      clone existed.
    
    """

    pkg_repo_base = os.path.basename(pkg_repo)
    pkg_path = os.path.join(packages_path, pkg_repo_base)

    ret = None
    
    if redo:
        try:
            shutil.rmtree(pkg_path)
        except FileNotFoundError:
            pass
    
    if not os.path.exists(pkg_path):
        logging.info('Cloning %s from %s', pkg_repo_base, pkg_repo)
        ret = os.system(f"{git} clone --bare {pkg_repo} {pkg_path}")
    
    return pkg_repo, ret

In [7]:
# Async loop over the package URLs, cloning each one and storing the return codes in a dictionary
results = {repo: ret for repo, ret in ThreadPool(processes=32).imap(clone_bare, pkg_download.values())}

2019-02-07 08:37:24,666: Thread-34: Cloning AstroTime.jl.git from https://github.com/JuliaAstro/AstroTime.jl.git
2019-02-07 08:37:24,721: Thread-15: Cloning ChainRecursive.jl.git from https://github.com/bramtayl/ChainRecursive.jl.git
2019-02-07 08:37:24,780: Thread-7: Cloning CombinatorialBandits.jl.git from https://github.com/dourouc05/CombinatorialBandits.jl.git
2019-02-07 08:37:24,990: Thread-20: Cloning HTSLIB.jl.git from https://github.com/OpenGene/HTSLIB.jl.git
2019-02-07 08:37:25,065: Thread-26: Cloning InteractiveChaos.jl.git from https://github.com/JuliaDynamics/InteractiveChaos.jl.git
2019-02-07 08:37:25,139: Thread-31: Cloning LibGit2.jl.git from https://github.com/jakebolewski/LibGit2.jl.git
2019-02-07 08:37:25,271: Thread-21: Cloning NumberedLines.jl.git from https://github.com/bramtayl/NumberedLines.jl.git


In [8]:
# Loop through the return codes. If a code was nonzero, try to clone again!
for pkg_repo, ret in results.items():
    if ret is not None and ret != 0:
        clone_bare(pkg_repo)

2019-02-07 08:37:56,768: MainThread: Cloning ChainRecursive.jl.git from https://github.com/bramtayl/ChainRecursive.jl.git
2019-02-07 08:38:01,302: MainThread: Cloning CombinatorialBandits.jl.git from https://github.com/dourouc05/CombinatorialBandits.jl.git
2019-02-07 08:38:06,136: MainThread: Cloning HTSLIB.jl.git from https://github.com/OpenGene/HTSLIB.jl.git
2019-02-07 08:38:10,309: MainThread: Cloning LibGit2.jl.git from https://github.com/jakebolewski/LibGit2.jl.git
2019-02-07 08:38:13,813: MainThread: Cloning NumberedLines.jl.git from https://github.com/bramtayl/NumberedLines.jl.git


In [9]:
def fetch(pkg_repo, packages_path=packages_path):

    """Performs a fetch in a local Git repository, whose location is
    determined from the remote URL.
    
    Parameters
    ----------
    pkg_repo : str
      The URL of the remote Git repository.
    packages_path : str
      The path to the local directory where the clone is located.
      
    Returns
    -------
    pkg_repo : str
      The input `pkg_repo` parameter.
    ret : int
      The return code from Git, or ``None`` if the clone doesn't exist.
    
    """

    pkg_repo_base = os.path.basename(pkg_repo)
    pkg_path = os.path.join(packages_path, pkg_repo_base)

    ret = None
    
    if os.path.exists(pkg_path):
        logging.info('Fetching in %s', pkg_path)
        pwd = os.getcwd()
        os.chdir(pkg_path)
        ret = os.system(f"{git} fetch")
        os.chdir(pwd)
    
    return pkg_repo, ret

In [10]:
# Async loop over the package URLs, fetching on each one to ensure we have the latest and the clone is good
results = {repo: ret for repo, ret in ThreadPool(processes=32).imap(fetch, pkg_download.values())}

2019-02-07 08:38:17,153: Thread-53: Fetching in K:\julia_packages\AMD.jl.git
2019-02-07 08:38:17,158: Thread-51: Fetching in K:\julia_packages\ASTInterpreter2.jl.git
2019-02-07 08:38:17,158: Thread-47: Fetching in K:\julia_packages\ASTInterpreter.jl.git
2019-02-07 08:38:17,173: Thread-49: Fetching in K:\julia_packages\AbstractLattices.jl.git
2019-02-07 08:38:17,158: Thread-42: Fetching in K:\julia_packages\ASE.jl.git
2019-02-07 08:38:17,187: Thread-57: Fetching in K:\julia_packages\AbstractPlotting.jl.git
2019-02-07 08:38:17,158: Thread-54: Fetching in K:\julia_packages\AWSSDB.jl.git
2019-02-07 08:38:17,159: Thread-61: Fetching in K:\julia_packages\AWSSES.jl.git
2019-02-07 08:38:17,159: Thread-65: Fetching in K:\julia_packages\AWSSQS.jl.git
2019-02-07 08:38:17,159: Thread-45: Fetching in K:\julia_packages\ACME.jl.git
2019-02-07 08:38:17,159: Thread-62: Fetching in K:\julia_packages\AWS.jl.git
2019-02-07 08:38:17,159: Thread-64: Fetching in K:\julia_packages\AWSSNS.jl.git
2019-02-07 08:

2019-02-07 08:39:33,713: Thread-49: Fetching in K:\julia_packages\Atom.jl.git
2019-02-07 08:39:33,751: Thread-52: Fetching in K:\julia_packages\Attrs.jl.git
2019-02-07 08:39:33,838: Thread-46: Fetching in K:\julia_packages\AudioIO.jl.git
2019-02-07 08:39:33,891: Thread-62: Fetching in K:\julia_packages\AuditoryFilters.jl.git
2019-02-07 08:39:33,891: Thread-68: Fetching in K:\julia_packages\AugmentedGaussianProcesses.jl.git
2019-02-07 08:39:35,341: Thread-72: Fetching in K:\julia_packages\Augmentor.jl.git
2019-02-07 08:39:36,027: Thread-61: Fetching in K:\julia_packages\Augur.jl.git
2019-02-07 08:39:37,084: Thread-70: Fetching in K:\julia_packages\Authorization.jl.git
2019-02-07 08:39:37,319: Thread-57: Fetching in K:\julia_packages\AutoDiffSource.jl.git
2019-02-07 08:39:37,611: Thread-58: Fetching in K:\julia_packages\AutoGrad.jl.git
2019-02-07 08:39:38,267: Thread-47: Fetching in K:\julia_packages\AutoHashEquals.jl.git
2019-02-07 08:39:38,284: Thread-71: Fetching in K:\julia_packages\

2019-02-07 08:40:43,492: Thread-52: Fetching in K:\julia_packages\BloomFilters.jl.git
2019-02-07 08:40:43,619: Thread-55: Fetching in K:\julia_packages\Blosc.jl.git
2019-02-07 08:40:43,744: Thread-63: Fetching in K:\julia_packages\BlossomV.jl.git
2019-02-07 08:40:44,352: Thread-72: Fetching in K:\julia_packages\BlsData.jl.git
2019-02-07 08:40:45,908: Thread-67: Fetching in K:\julia_packages\Bokeh.jl.git
2019-02-07 08:40:46,094: Thread-50: Fetching in K:\julia_packages\Boltzmann.jl.git
2019-02-07 08:40:46,122: Thread-57: Fetching in K:\julia_packages\BoltzmannMachines.jl.git
2019-02-07 08:40:46,451: Thread-58: Fetching in K:\julia_packages\BoltzmannMachinesPlots.jl.git
2019-02-07 08:40:47,364: Thread-53: Fetching in K:\julia_packages\BooSTjl.jl.git
2019-02-07 08:40:47,586: Thread-47: Fetching in K:\julia_packages\Bootstrap.jl.git
2019-02-07 08:40:47,956: Thread-54: Fetching in K:\julia_packages\BotCoreLCMTypes.jl.git
2019-02-07 08:40:48,130: Thread-56: Fetching in K:\julia_packages\Boun

2019-02-07 08:41:59,995: Thread-71: Fetching in K:\julia_packages\Changepoints.jl.git
2019-02-07 08:42:00,917: Thread-52: Fetching in K:\julia_packages\ChaosCommunications.jl.git
2019-02-07 08:42:01,088: Thread-54: Fetching in K:\julia_packages\ChaosTools.jl.git
2019-02-07 08:42:01,349: Thread-56: Fetching in K:\julia_packages\CharSetEncodings.jl.git
2019-02-07 08:42:01,477: Thread-41: Fetching in K:\julia_packages\Chatter.jl.git
2019-02-07 08:42:05,225: Thread-60: Fetching in K:\julia_packages\Checkers.jl.git
2019-02-07 08:42:05,294: Thread-55: Fetching in K:\julia_packages\Chemfiles.jl.git
2019-02-07 08:42:05,490: Thread-45: Fetching in K:\julia_packages\ChemicalKinetics.jl.git
2019-02-07 08:42:05,585: Thread-47: Fetching in K:\julia_packages\Chipmunk.jl.git
2019-02-07 08:42:06,449: Thread-69: Fetching in K:\julia_packages\CholmodSolve2.jl.git
2019-02-07 08:42:06,521: Thread-44: Fetching in K:\julia_packages\ChrBase.jl.git
2019-02-07 08:42:07,539: Thread-65: Fetching in K:\julia_pack

2019-02-07 08:43:12,512: Thread-54: Fetching in K:\julia_packages\Convertible.jl.git
2019-02-07 08:43:13,647: Thread-52: Fetching in K:\julia_packages\Convex.jl.git
2019-02-07 08:43:14,955: Thread-45: Fetching in K:\julia_packages\ConvexBodyProximityQueries.jl.git
2019-02-07 08:43:15,641: Thread-48: Fetching in K:\julia_packages\ConvexSwitch.jl.git
2019-02-07 08:43:16,627: Thread-56: Fetching in K:\julia_packages\Cookbook.jl.git
2019-02-07 08:43:17,728: Thread-44: Fetching in K:\julia_packages\CoordinateDescent.jl.git
2019-02-07 08:43:18,576: Thread-55: Fetching in K:\julia_packages\CoordinateTransformations.jl.git
2019-02-07 08:43:19,180: Thread-60: Fetching in K:\julia_packages\CoreNLP.jl.git
2019-02-07 08:43:19,438: Thread-69: Fetching in K:\julia_packages\CornerPlot.jl.git
2019-02-07 08:43:20,329: Thread-68: Fetching in K:\julia_packages\Cosmology.jl.git
2019-02-07 08:43:20,635: Thread-65: Fetching in K:\julia_packages\Coulter.jl.git
2019-02-07 08:43:22,004: Thread-49: Fetching in 

...

2019-02-07 09:04:04,984: Thread-45: Fetching in K:\julia_packages\ShiftedArrays.jl.git
2019-02-07 09:04:05,669: Thread-58: Fetching in K:\julia_packages\Shoco.jl.git
2019-02-07 09:04:05,822: Thread-50: Fetching in K:\julia_packages\ShowItLikeYouBuildIt.jl.git
2019-02-07 09:04:10,738: Thread-42: Fetching in K:\julia_packages\ShowSet.jl.git
2019-02-07 09:04:10,824: Thread-59: Fetching in K:\julia_packages\Showoff.jl.git
2019-02-07 09:04:11,602: Thread-53: Fetching in K:\julia_packages\Sigma.jl.git
2019-02-07 09:04:12,691: Thread-47: Fetching in K:\julia_packages\SignalView.jl.git
2019-02-07 09:04:12,712: Thread-44: Fetching in K:\julia_packages\SigmoidalProgramming.jl.git
2019-02-07 09:04:13,396: Thread-71: Fetching in K:\julia_packages\Signals.jl.git
2019-02-07 09:04:16,584: Thread-41: Fetching in K:\julia_packages\SignedDistanceFields.jl.git
2019-02-07 09:04:17,527: Thread-49: Fetching in K:\julia_packages\Silo.jl.git
2019-02-07 09:04:17,958: Thread-60: Fetching in K:\julia_packages\Si

2019-02-07 09:05:23,478: Thread-59: Fetching in K:\julia_packages\StateSpaceReconstruction.jl.git
2019-02-07 09:05:24,024: Thread-63: Fetching in K:\julia_packages\StateSpaceRoutines.jl.git
2019-02-07 09:05:24,212: Thread-47: Fetching in K:\julia_packages\StaticArrays.jl.git
2019-02-07 09:05:24,366: Thread-49: Fetching in K:\julia_packages\StatefulIterators.jl.git
2019-02-07 09:05:24,452: Thread-42: Fetching in K:\julia_packages\StaticGraphs.jl.git
2019-02-07 09:05:24,504: Thread-71: Fetching in K:\julia_packages\StaticLint.jl.git
2019-02-07 09:05:24,566: Thread-41: Fetching in K:\julia_packages\StaticNumbers.jl.git
2019-02-07 09:05:24,766: Thread-58: Fetching in K:\julia_packages\StaticPolynomials.jl.git
2019-02-07 09:05:25,034: Thread-46: Fetching in K:\julia_packages\StaticUnivariatePolynomials.jl.git
2019-02-07 09:05:27,534: Thread-51: Fetching in K:\julia_packages\StatisticalRethinking.jl.git
2019-02-07 09:05:27,961: Thread-60: Fetching in K:\julia_packages\Stats.jl.git
2019-02-07

2019-02-07 09:06:31,905: Thread-57: Fetching in K:\julia_packages\SyntaxTree.jl.git
2019-02-07 09:06:32,780: Thread-41: Fetching in K:\julia_packages\SynthesisFilters.jl.git
2019-02-07 09:06:34,826: Thread-68: Fetching in K:\julia_packages\SyntheticGrids.jl.git
2019-02-07 09:06:35,420: Thread-42: Fetching in K:\julia_packages\Syslogs.jl.git
2019-02-07 09:06:36,500: Thread-45: Fetching in K:\julia_packages\SystemImageBuilder.jl.git
2019-02-07 09:06:36,586: Thread-65: Fetching in K:\julia_packages\TOML.jl.git
2019-02-07 09:06:36,861: Thread-44: Fetching in K:\julia_packages\TSVD.jl.git
2019-02-07 09:06:37,683: Thread-64: Fetching in K:\julia_packages\TSne.jl.git
2019-02-07 09:06:38,836: Thread-69: Fetching in K:\julia_packages\TableSchema.jl.git
2019-02-07 09:06:39,236: Thread-70: Fetching in K:\julia_packages\TableShowUtils.jl.git
2019-02-07 09:06:41,601: Thread-66: Fetching in K:\julia_packages\TableTraits.jl.git
2019-02-07 09:06:42,073: Thread-43: Fetching in K:\julia_packages\TableTr

2019-02-07 09:07:44,554: Thread-68: Fetching in K:\julia_packages\Transit.jl.git
2019-02-07 09:07:48,095: Thread-66: Fetching in K:\julia_packages\Transpiler.jl.git
2019-02-07 09:07:48,182: Thread-55: Fetching in K:\julia_packages\TravelingSalesmanHeuristics.jl.git
2019-02-07 09:07:48,248: Thread-70: Fetching in K:\julia_packages\TreeView.jl.git
2019-02-07 09:07:48,345: Thread-43: Fetching in K:\julia_packages\TreeViews.jl.git
2019-02-07 09:07:49,404: Thread-54: Fetching in K:\julia_packages\Triangle.jl.git
2019-02-07 09:07:49,773: Thread-62: Fetching in K:\julia_packages\TriangleMesh.jl.git
2019-02-07 09:07:51,062: Thread-42: Fetching in K:\julia_packages\TropicalSemiring.jl.git
2019-02-07 09:07:51,313: Thread-61: Fetching in K:\julia_packages\TupleTools.jl.git
2019-02-07 09:07:53,874: Thread-48: Fetching in K:\julia_packages\Turing.jl.git
2019-02-07 09:07:54,097: Thread-44: Fetching in K:\julia_packages\TwentyFour.jl.git
2019-02-07 09:07:55,641: Thread-46: Fetching in K:\julia_packag

2019-02-07 09:08:56,866: Thread-43: Fetching in K:\julia_packages\WaveletMatrices.jl.git
2019-02-07 09:09:02,469: Thread-70: Fetching in K:\julia_packages\Wavelets.jl.git
2019-02-07 09:09:03,002: Thread-44: Fetching in K:\julia_packages\WeakRefStrings.jl.git
2019-02-07 09:09:06,384: Thread-61: Fetching in K:\julia_packages\Weave.jl.git
2019-02-07 09:09:06,955: Thread-62: Fetching in K:\julia_packages\WebAssembly.jl.git
2019-02-07 09:09:07,959: Thread-65: Fetching in K:\julia_packages\WebFuncs.jl.git
2019-02-07 09:09:07,983: Thread-47: Fetching in K:\julia_packages\WebIO.jl.git
2019-02-07 09:09:07,999: Thread-71: Fetching in K:\julia_packages\WebSockets.jl.git
2019-02-07 09:09:08,218: Thread-55: Fetching in K:\julia_packages\Weber.jl.git
2019-02-07 09:09:08,436: Thread-67: Fetching in K:\julia_packages\WeberCedrus.jl.git
2019-02-07 09:09:09,361: Thread-49: Fetching in K:\julia_packages\WeberDAQmx.jl.git
2019-02-07 09:09:09,495: Thread-51: Fetching in K:\julia_packages\WeightedArrays.jl.

In [11]:
# Loop through the return codes. If a code was nonzero, try to fetch again!
# Anything that fails at this point is either an invalid URL or we have bad creds!
for pkg_repo, ret in results.items():
    if ret is not None and ret != 0:
        clone_bare(pkg_repo, redo=True)

In [4]:
def onerror(func, path, exc_info):
    
    """Error handler for ``shutil.rmtree``.

    If the error is due to an access error (read only file)
    it attempts to add write permission and then retries.

    If the error is for another reason it re-raises the error.

    Usage : ``shutil.rmtree(path, onerror=onerror)``
    
    """
    
    import stat
    if not os.access(path, os.W_OK):
        # Is the error an access error ?
        os.chmod(path, stat.S_IWUSR)
        func(path)
    else:
        raise

In [3]:
# The Julia language repo needs to download some dependencies when it's built.
# This is normally done using "make -d deps getall", but we don't have GNU make
# on the IAS. So we have to walk the Makefiles ourselves and manuall download
# the dependencies so we can ingress them, too!

# Clone Julia from the bare repo
julia_repo = os.path.join(packages_path, 'julia.git')
julia_path = os.path.join(packages_path, 'julia')
if os.path.exists(julia_path):
    shutil.rmtree(julia_path, onerror=onerror)
if os.system(f"{git} clone {julia_repo} {julia_path}") != 0:
    raise RuntimeError("Clone of julia repo failed!")

In [ ]:
# Run GNU make to get the dependencies
pwd = os.getcwd()
os.chdir(os.path.join(julia_path, 'deps'))
ret = os.system(f"make getall")
os.chdir(pwd)
if ret != 0:
    raise RuntimeError("Getting Julia dependencies failed!")

In [ ]:
# Zip up the directory to transfer to NNPP
topdir = os.path.realpath(os.path.dirname(packages_path))
pwd = os.getcwd()
os.chdir(topdir)
ret = os.system(f"tar czf julia_packages.tar.gz julia_packages")
os.chdir(pwd)
if ret != 0:
    raise RuntimeError("Zipping Julia packages failed!")